<a href="https://colab.research.google.com/github/Kartikaggarwal98/Propagada-Detection/blob/master/propaganda_oversample_sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

In [0]:
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline,linear_model
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from keras.preprocessing import text, sequence
from keras import layers, models, optimizers
import keras
import re
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score


Using TensorFlow backend.


In [0]:
from nltk import word_tokenize
from nltk.corpus import stopwords

import nltk
nltk.download('stopwords')
nltk.download('punkt')

stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
PATH=Path("/content/drive/My Drive/emnlp/")

In [0]:
os.listdir(PATH)

['datasets', 'Data_Prepare.ipynb', 'traindf.csv']

In [0]:
df=pd.read_csv(PATH/'traindf.csv')

df=df.dropna() ## every alternate line is blank line

label_mapping={'propaganda':0,'non-propaganda':1} ## label encoding
df['labels']=df['labels'].apply(lambda x:label_mapping[x])
df=pd.concat([df,df[df['labels']==0].copy()],0).reset_index(drop=True)
df.head()

,Unnamed: 0,sentence,labels
0,0,US bloggers banned from entering UK,1
1,2,Two prominent US bloggers have been banned fro...,1
2,4,Pamela Geller and Robert Spencer co-founded an...,0
3,6,They were due to speak at an English Defence L...,1
4,8,A government spokesman said individuals whose ...,1


In [0]:
ds,labels=df['sentence'],df['labels']

In [0]:
np.unique(labels,return_counts=True)

(array([0, 1]), array([ 9440, 11577]))

In [0]:
X_train,X_test,y_train,y_test=train_test_split(ds,labels,test_size=0.2,random_state=42,)
X_train.shape,X_test.shape,y_train.shape,y_test.shape

((16813,), (4204,), (16813,), (4204,))

In [0]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
# word level tf-idf
tfidf_vect = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', max_features=5000)
# ngram level tf-idf 
tfidf_vect_ngram = TfidfVectorizer(analyzer='word', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)
# characters level tf-idf
tfidf_vect_ngram_chars = TfidfVectorizer(analyzer='char', token_pattern=r'\w{1,}', ngram_range=(2,3), max_features=5000)

In [0]:
print('===========================   Naive Bayes =====================')


vectorizers=[(count_vect,'count_vectorizer'),(tfidf_vect,'tfidf_vectorizer_word'),(tfidf_vect_ngram,'tfidf_vectorizer_word_ngram'),(tfidf_vect_ngram_chars,'tfidf_vectorizer_ngram_chars')]

#Naive Bayes for all features:
for vectorizer in vectorizers:
  print(vectorizer[1])
  clf = Pipeline([
    ('vect',vectorizer[0]),
    ('clf', MultinomialNB()),
  ])
  clf = clf.fit(X_train, y_train)
  predictions=clf.predict_proba(X_test)
  test_preds=np.argmax(predictions,axis=1)

  auc = classification_report(y_test, test_preds)
  print (auc)


===========================   Naive Bayes =====================
count_vectorizer
              precision    recall  f1-score   support

           0       0.67      0.79      0.72      1851
           1       0.80      0.69      0.75      2353

    accuracy                           0.73      4204
   macro avg       0.74      0.74      0.73      4204
weighted avg       0.74      0.73      0.74      4204

tfidf_vectorizer_word
              precision    recall  f1-score   support

           0       0.67      0.66      0.67      1851
           1       0.74      0.74      0.74      2353

    accuracy                           0.71      4204
   macro avg       0.70      0.70      0.70      4204
weighted avg       0.71      0.71      0.71      4204

tfidf_vectorizer_word_ngram
              precision    recall  f1-score   support

           0       0.62      0.64      0.63      1851
           1       0.71      0.69      0.70      2353

    accuracy                           0.67      42

In [0]:
print('===========================   Logistic Regression =====================')


vectorizers=[(count_vect,'count_vectorizer'),(tfidf_vect,'tfidf_vectorizer_word'),(tfidf_vect_ngram,'tfidf_vectorizer_word_ngram'),(tfidf_vect_ngram_chars,'tfidf_vectorizer_ngram_chars')]

for vectorizer in vectorizers:
  print(vectorizer[1])
  clf = Pipeline([
    ('vect',vectorizer[0]),
    ('clf', linear_model.LogisticRegression(multi_class='auto',solver='lbfgs')),
  ])
  clf = clf.fit(X_train, y_train)
  predictions=clf.predict_proba(X_test)
  test_preds=np.argmax(predictions,axis=1)

  auc = classification_report(y_test, test_preds)
  print (auc)

===========================   Logistic Regression =====================
count_vectorizer


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


              precision    recall  f1-score   support

           0       0.75      0.76      0.75      1851
           1       0.81      0.80      0.80      2353

    accuracy                           0.78      4204
   macro avg       0.78      0.78      0.78      4204
weighted avg       0.78      0.78      0.78      4204

tfidf_vectorizer_word
              precision    recall  f1-score   support

           0       0.69      0.66      0.67      1851
           1       0.74      0.77      0.75      2353

    accuracy                           0.72      4204
   macro avg       0.72      0.71      0.71      4204
weighted avg       0.72      0.72      0.72      4204

tfidf_vectorizer_word_ngram
              precision    recall  f1-score   support

           0       0.65      0.59      0.62      1851
           1       0.70      0.75      0.72      2353

    accuracy                           0.68      4204
   macro avg       0.68      0.67      0.67      4204
weighted avg       0.68 

In [0]:
print('===========================   SVM =====================')


vectorizers=[(count_vect,'count_vectorizer'),(tfidf_vect,'tfidf_vectorizer_word'),(tfidf_vect_ngram,'tfidf_vectorizer_word_ngram'),(tfidf_vect_ngram_chars,'tfidf_vectorizer_ngram_chars')]

# SVM on count vectors: SVM Classifier Pipeline on word count vector
for vectorizer in vectorizers:
  print(vectorizer[1])
  clf = Pipeline([
    ('vect',vectorizer[0]),
    ('clf', SVC(gamma='scale',probability=True)),
  ])
  clf = clf.fit(X_train, y_train)
  predictions=clf.predict_proba(X_test)
  test_preds=np.argmax(predictions,axis=1)

  auc = classification_report(y_test, test_preds)
  print (auc)

===========================   SVM =====================
count_vectorizer
              precision    recall  f1-score   support

           0       0.76      0.73      0.75      1851
           1       0.79      0.82      0.81      2353

    accuracy                           0.78      4204
   macro avg       0.78      0.78      0.78      4204
weighted avg       0.78      0.78      0.78      4204

tfidf_vectorizer_word
              precision    recall  f1-score   support

           0       0.80      0.80      0.80      1851
           1       0.84      0.84      0.84      2353

    accuracy                           0.82      4204
   macro avg       0.82      0.82      0.82      4204
weighted avg       0.82      0.82      0.82      4204

tfidf_vectorizer_word_ngram
              precision    recall  f1-score   support

           0       0.76      0.75      0.75      1851
           1       0.81      0.81      0.81      2353

    accuracy                           0.78      4204
   ma

Dev Set Prediction

In [0]:
train_folder = "/content/drive/My Drive/emnlp/datasets/train-articles" # check that the path to the datasets folder is correct,
dev_folder = "/content/drive/My Drive/emnlp/datasets/dev-articles"     # if not adjust these variables accordingly
train_labels_folder = "/content/drive/My Drive/emnlp/datasets/train-labels-SLC"
dev_template_labels_file = "/content/drive/My Drive/emnlp/datasets/dev.template-output-SLC.out"
task_SLC_output_file = "/content/drive/My Drive/emnlp/baseline-output-SLC1.txt"

In [0]:
import glob
import os.path
import numpy as np
import sys


def read_articles_from_file_list(folder_name, file_pattern="*.txt"):
    
    file_list = glob.glob(os.path.join(folder_name, file_pattern))
    article_id_list, sentence_id_list, sentence_list = ([], [], [])
    for filename in sorted(file_list):
        article_id = os.path.basename(filename).split(".")[0][7:]
        with open(filename, "r", encoding="utf-8") as f:
            for sentence_id, row in enumerate(f.readlines(), 1):
                sentence_list.append(row.rstrip())
                article_id_list.append(article_id)
                sentence_id_list.append(str(sentence_id))

    return article_id_list, sentence_id_list, sentence_list


def are_ids_aligned(article_id_list, sentence_id_list,
                    reference_article_id_list, reference_sentence_id_list):
    """
    check whether the two lists of ids of the articles and the sentences are aligned
    """
    for art, ref_art, sent, ref_sent in zip(article_id_list, reference_article_id_list,
                                            sentence_id_list, reference_sentence_id_list):
        if art != ref_art:
            print("ERROR: article ids do not match: article id = %s, reference article id = %s"%(art, ref_art))
            return False
        if sent != ref_sent:
            print("ERROR: sentence ids do not match: article id:%s,%s sentence id:%s,%s" %(art, ref_art, sent, ref_sent))
            return False
    return True


def read_predictions_from_file(filename):
   
    articles_id, sentence_id_list, gold_labels = ([], [], [])
    with open(filename, "r") as f:
        for row in f.readlines():
            article_id, sentence_id, gold_label = row.rstrip().split("\t")
            articles_id.append(article_id)
            sentence_id_list.append(sentence_id)
            gold_labels.append(gold_label)
    return articles_id, sentence_id_list, gold_labels


def read_predictions_from_file_list(folder_name, file_pattern):
    gold_file_list = glob.glob(os.path.join(folder_name, file_pattern))
    articles_id, sentence_id_list, gold_labels = ([], [], [])
    for filename in sorted(gold_file_list):
        art_ids, sent_ids, golds = read_predictions_from_file(filename)
        articles_id += art_ids
        sentence_id_list += sent_ids
        gold_labels += golds
    return articles_id, sentence_id_list, gold_labels


In [0]:
os.listdir('/content/drive/My Drive/emnlp/datasets/')

['dev-articles',
 'train-labels-FLC',
 'dev-template-output-SLC',
 'README.md',
 'train.task-SLC.labels',
 'train-articles',
 'train-labels-SLC',
 'dev.template-output-SLC.out']

In [0]:
# reading data from the development set
dev_article_id_list, dev_sentence_id_list, dev_sentence_list = read_articles_from_file_list(dev_folder)
reference_articles_id, reference_sentence_id_list, dev_labels = read_predictions_from_file(dev_template_labels_file)

In [0]:
len(dev_article_id_list),len(reference_articles_id)

(2235, 2235)

In [0]:
if not are_ids_aligned(dev_article_id_list, dev_sentence_id_list, reference_articles_id, reference_sentence_id_list):
    sys.exit("Exiting: development set article ids and gold labels are not aligned")

# computing the predictions on the development set
dev=dev_sentence_list

In [0]:
clf = Pipeline([
    ('vect',tfidf_vect),
    ('clf', SVC(gamma='scale',probability=True)),
  ])
clf = clf.fit(X_train, y_train)
predictions=clf.predict_proba(X_test)
test_preds=np.argmax(predictions,axis=1)

auc = classification_report(y_test, test_preds)
print (auc)

              precision    recall  f1-score   support

           0       0.80      0.80      0.80      1851
           1       0.84      0.84      0.84      2353

    accuracy                           0.82      4204
   macro avg       0.82      0.82      0.82      4204
weighted avg       0.82      0.82      0.82      4204



In [0]:
preds = clf.predict(dev)

In [0]:
label_inverse_mapping={0:'propaganda',1:'non-propaganda'} ## label encoding

predictions=np.array([label_inverse_mapping[x] for x in preds])

In [0]:
predictions.shape,preds.shape

((2235,), (2235,))

In [0]:
np.unique(predictions,return_counts=True),np.unique(preds,return_counts=True)

((array(['non-propaganda', 'propaganda'], dtype='<U14'), array([1581,  654])),
 (array([0, 1]), array([ 654, 1581])))

In [0]:
task_SLC_output_file = "/content/drive/My Drive/emnlp/submissions/SLC1.txt"
# writing predictions to file
with open(task_SLC_output_file, "w") as fout:
    for article_id, sentence_id, prediction in zip(dev_article_id_list, dev_sentence_id_list, predictions):
        fout.write("%s\t%s\t%s\n" % (article_id, sentence_id, prediction))
print("Predictions written to file " + task_SLC_output_file)

Predictions written to file /content/drive/My Drive/emnlp/submissions/SLC1.txt
